In [19]:
import pandas as pd
import pandas_ta as ta

In [20]:
macro = pd.read_csv('macro_monthly.csv')

In [21]:
macro['DATE'] = pd.to_datetime(macro['DATE'])
macro.set_index('DATE', inplace = True)

In [22]:
#Personal Savings Rate, M2 Money Supply, Core Consumer Price Index, Disposable Personal Income, Treasury Rate, Federal Funds Rate, Industrial Production
macro = macro[['psr','m2','ccpi', 'dspic', 'ir', 'ffer']]

In [23]:
dates = pd.date_range(macro.index.min(), '2021-12-31', name = 'date')

In [24]:
macro = macro.reindex(dates, method = 'ffill')

In [25]:
sp500 = pd.read_csv('sp500_index.csv')

In [26]:
sp500.rename(columns = {'Date':'date','S&P500':'close'}, inplace = True)
sp500.date = pd.to_datetime(sp500.date)
sp500.set_index('date')

,close
date,
2012-01-17,1293.67
2012-01-18,1308.04
2012-01-19,1314.50
2012-01-20,1315.38
2012-01-23,1316.00
...,...
2022-01-07,4677.03
2022-01-10,4670.29
2022-01-11,4713.07


In [27]:
sp500['return'] = (sp500.close / sp500.close.shift(1)) - 1

In [28]:
bbands = ta.bbands(sp500.close)
bbands.drop(['BBL_5_2.0', 'BBM_5_2.0', 'BBU_5_2.0'], axis = 1, inplace = True)

In [29]:
sp500 = sp500.join(bbands).set_index('date')

In [30]:
for i in [7,22,50,252]:
    ema_name = f'ema_{i}'
    sp500[ema_name] = ta.ema(sp500.close, i)

In [31]:
sp500 = sp500.add_prefix('sp500_')

In [32]:
macro = macro.join(sp500)

In [35]:
macro.dropna()

,psr,m2,ccpi,dspic,ir,ffer,sp500_close,sp500_return,sp500_BBB_5_2.0,sp500_BBP_5_2.0,sp500_ema_7,sp500_ema_22,sp500_ema_50,sp500_ema_252
date,,,,,,,,,,,,,,
2013-01-16,5.9,10513.60,232.229,12258.9,1.914762,0.142581,1472.63,0.000197,0.182912,0.747363,1467.755063,1449.966375,1434.433098,1386.521468
2013-01-17,5.9,10513.60,232.229,12258.9,1.914762,0.142581,1480.94,0.005643,0.995422,0.991622,1471.051297,1452.659734,1436.256898,1387.267860
2013-01-18,5.9,10513.60,232.229,12258.9,1.914762,0.142581,1485.98,0.003403,1.606091,0.899171,1474.783473,1455.557148,1438.206824,1388.048193
2013-01-22,5.9,10513.60,232.229,12258.9,1.914762,0.142581,1492.56,0.004428,2.104172,0.874513,1479.227605,1458.774788,1440.338321,1388.874373
2013-01-23,5.9,10513.60,232.229,12258.9,1.914762,0.142581,1494.81,0.001507,2.164780,0.793140,1483.123203,1461.908284,1442.474465,1389.711809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,7.3,21152.35,281.695,15425.2,1.582500,0.079032,4791.19,0.013839,6.391112,0.850694,4701.085820,4663.667557,4618.076580,4251.738549
2021-12-28,7.3,21152.35,281.695,15425.2,1.582500,0.079032,4786.35,-0.001010,4.566631,0.761702,4722.401865,4674.335595,4624.675537,4255.964726
2021-12-29,7.3,21152.35,281.695,15425.2,1.582500,0.079032,4793.06,0.001402,3.350788,0.716180,4740.066399,4684.659456,4631.278850,4260.210539


In [36]:
macro.to_csv('marketsentiment.csv')